In [2]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

This will load results for one of four datasets: random self instance/random unitaries for 3q/4q circuits.

In [3]:
results_paths = [
    '3q_random_self_rs',
    '3q_random_unitary_rs',
    '4q_random_self_rs',
    '4q_random_unitary_rs'
]

def load_dataset(n):
    assert 0<=n<=3, 'There are only four datasets'
    results = []
    for i in range(10):
        with open(f'{results_paths[n]}{i}', 'rb')  as f:
            results.append(pickle.load(f))
    return results

Here is the code to reproduce any of the charts from the paper.

In [3]:
results[0][0]['num_qubits']
results[0][0]['target_type']

NameError: name 'results' is not defined

In [ ]:
def mean_and_std(results):
    mean_list = []
    std_list = []
    for i in range(len(results[0])):
        sr_list = np.array([res[i]['success_ratio'] for res in results])
        mean_list.append(np.mean(sr_list))
        std_list.append(np.std(sr_list))
    
    return mean_list, std_list    


# Change index to plot a different dataset.
results = load_dataset(0)

# Compute mean and standard deviation at each gate count      
mean_list, std_list = mean_and_std(results)

# Do the plotting.
plt.errorbar(range(len(mean_list)), mean_list, yerr=std_list, fmt='o', capsize=10);

plt.title(f"num qubits: {results[0][0]['num_qubits']}, target type: {results[0][0]['target_type']}")
plt.yscale('log')
plt.xticks(range(len(mean_list)), rotation=70)
plt.ylabel('Success Ratio')
plt.xlabel('Number of CZ gates')

Here let's look at a single point from such a dataset: with a fixed target and fixed number of CZ gates.

In [ ]:
dataset_i = 3
target_index = 2
x_index = 4 # Equal to gate count for 3q circuits, equal to 
res = load_dataset(dataset_i)[target_index][x_index]

print('num qubits:', res['num_qubits'])
print('template gate count:', res['num_cz_gates'])
print('success ratio:', res['success_ratio'])
all_best_losses = res['losses']
all_best_losses = [float(l) for l in all_best_losses]

List `all_best_losses` contains a list of all minimal losses that were reached for a 1000 initial conditions. Below we plot a sample histogram and specify how the success ratio was computed.

In [ ]:
loss_cutoff = 1e-4

best_loss_overall = min(all_best_losses)
loss_threshold = best_loss_overall+loss_cutoff

success_ratio = sum(np.array(all_best_losses) < loss_threshold)/len(all_best_losses)
print(f'succes ratio :{success_ratio}')
plt.axvline(loss_threshold, color='red', linestyle='--')
plt.hist(all_best_losses, bins=100);